<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0


    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-11 09:41:57
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.42 C
-------------------
Today PnL: 41.30 K (0.29%)
Current PnL: -17.58 L (-11.83%)
CY Booked + Current PnL: -6.55 L (-4.41%)
-------------------
Total profit:  2.10 L
Total loss:  -19.68 L
-------------------
Total Booked + Current PnL: 20.43 L (16.79%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.79%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.23 L (54.53%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.5%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.20,60.0,H-LC,11.20,173393.0,17216.0,8878.0,-0.60,11.02,5.12,16.71,22.0,1.94,1.33,30.38,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,57.0,M-LC,4.28,216823.0,15344.0,11882.0,1.16,7.62,5.48,13.51,60.0,1.29,1.66,21.17,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,106.19,63.0,M-SC,5.26,88818.0,-11959.0,12053.0,0.21,-11.87,13.57,0.09,245.0,-0.99,0.68,16.69,OX40N,NTT,DURABLES
41,ITC,452.00,-37.08,56.0,H-LC,3.03,202544.0,2406.0,18492.0,0.28,1.20,9.13,10.44,4.0,0.13,1.55,6.83,X40,NTT,FMCG
49,NATIONALUM,247.44,-43.66,69.0,H-MC,3.42,112477.0,11973.0,18671.0,1.48,11.91,16.60,30.49,79.0,0.64,0.86,48.18,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,42.74,62.0,M-MC,6.98,225065.0,103.0,167561.0,0.76,0.05,74.45,74.53,192.0,0.00,1.72,32.31,XY24,BTT,STEEL
4,ANGELONE,3033.00,7.65,39.0,H-SC,6.92,190658.0,-348.0,58055.0,-0.49,-0.18,30.45,30.21,157.0,-0.01,1.46,20.66,X40N,NTT,FINANCE
35,ICICIPRULI,790.00,-21.00,41.0,H-MC,1.98,135849.0,61.0,42697.0,-0.19,0.05,31.43,31.48,107.0,0.00,1.04,12.17,X40,ATH,INSURANCE
34,ICICIGI,2260.25,-20.81,42.0,H-MC,3.22,136338.0,205.0,30921.0,-0.69,0.15,22.68,22.86,91.0,0.01,1.04,12.78,X40,ATH,INSURANCE
51,PGHH,17973.08,-31.08,50.0,H-MC,4.62,199455.0,-1365.0,70148.0,0.32,-0.68,35.17,34.25,80.0,-0.02,1.53,4.48,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.98,39.0,H-LC,6.30,302859.0,-12983.0,125656.0,-0.23,-4.11,41.49,35.67,5.0,-0.10,2.32,9.17,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.31,44.0,H-LC,13.63,218330.0,924.0,52574.0,0.60,0.43,24.08,24.61,48.0,0.02,1.67,55.48,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.09,46.0,H-LC,8.17,172788.0,-8971.0,44216.0,0.42,-4.94,25.59,19.39,86.0,-0.20,1.32,2.81,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,47.0,H-LC,10.17,229754.0,-12170.0,75451.0,-0.03,-5.03,32.84,26.15,8.0,-0.16,1.76,8.57,X40,ATH,IT
56,RELIANCE,1533.00,-14.04,48.0,H-LC,6.30,215452.0,5086.0,23700.0,0.30,2.42,11.00,13.68,37.0,0.21,1.65,19.34,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-4.62,55.0,H-LC,1.28,236272.0,-9879.0,89075.0,-1.06,-4.01,37.70,32.18,16.0,-0.11,1.81,30.58,X200,ATH,IT
41,ITC,452.00,-37.08,56.0,H-LC,3.03,202544.0,2406.0,18492.0,0.28,1.20,9.13,10.44,4.0,0.13,1.55,6.83,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.46,62.0,H-LC,3.33,284708.0,19282.0,37496.0,-0.07,7.26,13.17,21.40,11.0,0.51,2.18,15.61,XY25,NTT,FMCG
20,CIPLA,1795.00,-20.12,50.0,H-LC,5.93,213199.0,8699.0,32726.0,-0.49,4.25,15.35,20.25,10.0,0.27,1.63,13.36,X40N,BTT,PHARMA
83,VBL,671.64,-15.98,39.0,H-LC,6.30,302859.0,-12983.0,125656.0,-0.23,-4.11,41.49,35.67,5.0,-0.10,2.32,9.17,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GREENPANEL,537.0,228.81,71.0,M-SC,6.49,146717.0,-36361.0,115334.0,10.21,-19.86,78.61,43.14,230.0,-0.32,1.12,38.01,XY24,NTT,MISC
12,BANKINDIA,190.0,-27.95,69.0,H-MC,5.55,185375.0,5567.0,106850.0,2.59,3.10,57.64,62.52,88.0,0.05,1.42,38.24,XR,NTT,BANKS
79,UNIONBANK,163.0,-14.73,59.0,M-LC,8.76,154336.0,13496.0,30991.0,2.44,9.58,20.08,31.59,66.0,0.44,1.18,38.61,XY24,NTT,BANKS
57,REPCOHOME,880.0,-60.51,51.0,H-SC,7.95,147141.0,-57857.0,203099.0,2.34,-28.22,138.03,70.85,134.0,-0.28,1.13,20.88,XY24,NTT,FINANCE
39,INFY,2275.0,-16.37,58.0,H-LC,10.11,326444.0,13556.0,158129.0,1.88,4.33,48.44,54.87,3.0,0.09,2.50,11.27,X40,BTT,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,101.87,64.0,H-SC,12.69,127710.0,-14769.0,34290.0,-2.17,-10.37,26.85,13.70,163.0,-0.43,0.98,50.73,OX40N,NTT,BANKS
82,VALIANTORG,838.0,-326.45,55.0,H-SC,8.42,96200.0,-37086.0,121683.0,-1.95,-27.82,126.49,63.47,139.0,-0.30,0.74,61.26,XR,NTT,CHEMICALS
67,SONACOMS,1006.0,-37.48,47.0,M-SC,11.92,77875.0,-23284.0,98177.0,-1.12,-23.02,126.07,74.03,202.0,-0.24,0.60,9.81,AR,BTT,AUTO
47,LTIM,7230.2,-4.62,55.0,H-LC,1.28,236272.0,-9879.0,89075.0,-1.06,-4.01,37.70,32.18,16.0,-0.11,1.81,30.58,X200,ATH,IT
81,VAIBHAVGBL,521.0,53.72,62.0,H-SC,10.18,102480.0,-49511.0,123632.0,-1.04,-32.58,120.64,48.77,125.0,-0.40,0.78,19.28,XR,NTT,JEWELLERY


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,101.87,64.0,H-SC,12.69,127710.0,-14769.0,34290.0,-2.17,-10.37,26.85,13.70,163.0,-0.43,0.98,50.73,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,106.19,63.0,M-SC,5.26,88818.0,-11959.0,12053.0,0.21,-11.87,13.57,0.09,245.0,-0.99,0.68,16.69,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,55.0,H-SC,2.31,220509.0,-49158.0,85491.0,0.29,-18.23,38.77,13.47,138.0,-0.58,1.69,11.53,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,114.84,50.0,M-SC,2.76,141038.0,-33521.0,33553.0,0.51,-19.20,23.79,0.02,221.0,-1.00,1.08,22.68,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,45.0,H-SC,2.23,114102.0,-29701.0,56446.0,-0.55,-20.65,49.47,18.60,149.0,-0.53,0.87,25.91,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.44,54.0,H-LC,13.70,293788.0,-52168.0,176038.0,0.50,-15.08,59.92,35.80,1.0,-0.30,2.25,4.08,X40,BTT,IT
39,INFY,2275.00,-16.37,58.0,H-LC,10.11,326444.0,13556.0,158129.0,1.88,4.33,48.44,54.87,3.0,0.09,2.50,11.27,X40,BTT,IT
41,ITC,452.00,-37.08,56.0,H-LC,3.03,202544.0,2406.0,18492.0,0.28,1.20,9.13,10.44,4.0,0.13,1.55,6.83,X40,NTT,FMCG
83,VBL,671.64,-15.98,39.0,H-LC,6.30,302859.0,-12983.0,125656.0,-0.23,-4.11,41.49,35.67,5.0,-0.10,2.32,9.17,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,52.0,H-LC,12.29,249533.0,-12086.0,131304.0,0.71,-4.62,52.62,45.57,7.0,-0.09,1.91,6.18,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,42.74,62.0,M-MC,6.98,225065.0,103.0,167561.0,0.76,0.05,74.45,74.53,192.0,0.00,1.72,32.31,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.02,52.0,H-MC,3.73,188210.0,23540.0,44436.0,-0.40,14.30,23.61,41.28,89.0,0.53,1.44,23.42,X40N,ATH,AC
84,VOLTAS,1918.49,-0.94,53.0,H-MC,3.17,207900.0,16158.0,79875.0,-0.64,8.43,38.42,50.08,99.0,0.20,1.59,15.80,XY25,ATH,AC
24,DABUR,735.00,-2.58,60.0,H-MC,6.00,212062.0,15034.0,74582.0,-0.69,7.63,35.17,45.49,102.0,0.20,1.62,21.67,XY24,BTT,FMCG
18,CAMS,5211.76,-3.61,56.0,H-SC,4.73,109589.0,7585.0,36340.0,0.59,7.44,33.16,43.06,122.0,0.21,0.84,25.83,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-43.66,69.0,H-MC,3.42,112477.0,11973.0,18671.0,1.48,11.91,16.60,30.49,79.0,0.64,0.86,48.18,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.02,64.0,M-SC,1.81,103162.0,11664.0,67086.0,0.42,12.75,65.03,86.07,223.0,0.17,0.79,49.81,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.35,56.0,H-SC,12.19,138881.0,15545.0,118216.0,1.71,12.60,85.12,108.45,119.0,0.13,1.06,37.77,AR,ATH,MISC
86,WIPRO,420.00,-10.62,56.0,M-LC,6.26,157288.0,6343.0,103118.0,-0.81,4.20,65.56,72.51,53.0,0.06,1.20,10.48,XR,NTT,IT
12,BANKINDIA,190.00,-27.95,69.0,H-MC,5.55,185375.0,5567.0,106850.0,2.59,3.10,57.64,62.52,88.0,0.05,1.42,38.24,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6733.33,52.0,L-SC,5.97,74951.0,-18859.0,95480.0,0.37,-20.10,127.39,81.67,269.0,-0.20,0.57,45.15,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-4.00,47.0,H-SC,14.34,89414.0,-11549.0,106081.0,0.51,-11.44,118.64,93.63,148.0,-0.11,0.68,33.18,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,58.0,L-SC,43.45,88992.0,-24557.0,64608.0,-0.64,-21.63,72.60,35.27,268.0,-0.38,0.68,117.81,XR,NTT,HOTELS
48,MASFIN,398.61,-16.89,55.0,H-SC,13.05,94365.0,-3615.0,25214.0,0.05,-3.69,26.72,22.05,152.0,-0.14,0.72,37.17,XR,ATH,FINANCE
82,VALIANTORG,838.00,-326.45,55.0,H-SC,8.42,96200.0,-37086.0,121683.0,-1.95,-27.82,126.49,63.47,139.0,-0.30,0.74,61.26,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-39.64,58.0,L-SC,43.45,88992.0,-24557.0,64608.0,-0.64,-21.63,72.60,35.27,268.0,-0.38,0.68,117.81,XR,NTT,HOTELS
60,SAMMAANCAP,326.0,-171.59,64.0,M-SC,3.01,82464.0,-19746.0,113132.0,0.51,-19.32,137.19,91.37,208.0,-0.17,0.63,33.87,XY25,NTT,FINANCE
8,ATULAUTO,844.0,3651.35,60.0,M-SC,7.45,117009.0,-30359.0,82175.0,-0.13,-20.60,70.23,35.16,236.0,-0.37,0.90,19.51,XY24,NTT,AUTO
75,TITAGARH,1548.0,-3.85,62.0,H-SC,6.51,157692.0,-34323.0,113207.0,-0.85,-17.88,71.79,41.08,158.0,-0.30,1.21,33.62,XY24,NTT,ENGINEERING
64,SHALBY,327.0,1083.41,73.0,M-SC,1.85,161053.0,-20090.0,65565.0,-0.73,-11.09,40.71,25.10,235.0,-0.31,1.23,29.07,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.29
1,25,44.30
2,50,75.12


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.90
LC    35.39
MC    23.69
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.77
X40      16.04
X40N     13.08
XY25     10.25
XR        9.53
AR        8.40
OX40N     7.73
X200      1.81
MH        1.79
X5K       1.46
SR        1.12
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.81
H-SC    24.98
H-MC    20.60
M-SC    14.26
M-LC     7.47
M-MC     2.76
L-SC     1.66
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,12.95,-15.23,73.57
FMCG,12.26,-3.64,38.78
FINANCE,9.08,-17.83,60.20
BANKS,7.84,-11.41,58.71
MISC,6.17,-14.17,79.61
PAINTS,6.14,-15.07,40.83
ELECTRICAL,5.84,-7.76,67.15
AC,3.78,1.88,31.34
AUTO,3.47,-16.19,68.59


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2706156.0,22
AR,1174997.0,9
XR,1047963.0,13
X40,871058.0,10
X40N,567106.0,10
OX40N,449264.0,10
XY25,412480.0,7
SR,196726.0,2
X5K,127437.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2759505.0,28
M-SC,1384687.0,17
H-LC,1362778.0,15
H-MC,1152194.0,15
M-LC,403026.0,5
M-MC,317374.0,2
L-SC,244321.0,3
L-MC,59754.0,1
L-LC,39501.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,911125.0,6
M-SC,XY24,802243.0,7
H-SC,AR,644732.0,5
H-LC,X40,578211.0,5
H-SC,XR,504062.0,6
H-MC,XY24,462251.0,4
H-LC,AR,370246.0,2
M-MC,XY24,317374.0,2
H-SC,X40N,246732.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
